In [ ]:
import numpy
import pandas as pd
import yfinance as yf


# Movement Types

# 1. Stocks rise from current day's close to next day's open '/' movement
# 2. Stocks rise on a given day from Open to Close '//'
# 3. Stocks fall from current day's close to next day's open '\'
# 4. Stocks fall on a given day from Open to Close '\\'

'''
For a given stock, 

1. we want to have signal for if we want to buy the stock on a close of particular day
2. we want to hold it the next day or sell at open

'''




In [ ]:
df = yf.download("AMZN", start="2021-05-7", end="2021-08-14", interval="1d")

In [4]:
df['ClosePrev'] = df['Close'].shift(1)
df['OpenNext'] = df['Open'].shift(-1)


In [5]:
# num_stocks = 10
df['Prof'] = df['Open']-df['ClosePrev']
df['openProfPercent'] = 10000*(df['Prof']/df['ClosePrev'])
df['isHighMoreThanClose'] = df['High'] > df['ClosePrev']
df['best'] = 10000*((df['High'] - df['ClosePrev']))/df['ClosePrev']
df['worked'] = df['isHighMoreThanClose'] | (df['Prof'] > 0)
df['cross'] = (df['High'] > df['ClosePrev']) & ( df['ClosePrev'] > df['Open'])
df['sellEODProf'] = 10000*((df['Close'] - df['ClosePrev']))/df['ClosePrev']
df['% Change'] = 100*((df['Close'] - df['ClosePrev']))/df['ClosePrev']

df['strat%'] = df['openProfPercent']
df.loc[(df['Prof'] < 0) & (~df['cross']), 'strat%'] = df['sellEODProf']
df.loc[(df['Prof'] < 0) & (df['cross']), 'strat%'] = 0
# df.loc[(df['Prof'] > 0) & (df['sellEODProf'] < 0), 'movement'] = "\\"
df.loc[(df['Low'] < df['ClosePrev']), 'isBelowPrevClose'] = True
df.loc[(df['Low'] > df['ClosePrev']), 'isBelowPrevClose'] = False

# df.loc[(df['OpenNext'] > df['ClosePrev']) & df['isBelowPrevClose'], 'movement'] = "/"
# df.loc[(df['Open'] > df['ClosePrev']) & (df['Close'] < df['OpenNext']), 'movement'] = "\\"
# df.loc[(df['Close'] < df['OpenNext']), 'movementCloseToOpen'] = "/"
# df.loc[(df['Close'] > df['OpenNext']), 'movementCloseToOpen'] = "\\"
# df.loc[(df['Open'] < df['Close']), 'movementOpenToClose(Day)'] = "//"
# df.loc[(df['Open'] > df['Close']), 'movementOpenToClose(Day)'] = "\\\\"
df.loc[(df['OpenNext'] - df['Open']) > (df['OpenNext'] - df['Close']), 'Action'] = 'BuyAtOpen'
df.loc[(df['OpenNext'] - df['Open']) < (df['OpenNext'] - df['Close']), 'Action'] = 'BuyAtClose'


df.loc[(df['Action'] == 'BuyAtOpen') & (df['OpenNext'] - df['Open'] < 0), 'Action'] = 'NA'
df.loc[(df['Action'] == 'BuyAtClose') & (df['OpenNext'] - df['Close'] < 0), 'Action'] = 'NA'

# df['shiftedChange'] = df['% Change'].shift(1)
# df['CumChange'] = df['shiftedChange'] + df['% Change']

# df.loc[(df['shiftedChange'] > 0) & (df['% Change'] > 0), 'CumChange'] = df['CumChange'] + df['% Change']
# df.loc[(df['shiftedChange'] < 0) & (df['% Change'] < 0), 'CumChange'] = df['CumChange'] + df['% Change']

# df.loc[(df['shiftedChange'] > 0) & (df['% Change'] < 0), 'CumChange'] = df['% Change']
# df.loc[(df['shiftedChange'] < 0) & (df['% Change'] > 0), 'CumChange'] = df['% Change']





# df.loc[(df['Prof'] < df['sellEODProf']), 'movement'] = "//"
# df.loc[(df['Prof'] > 0) & (df['sellEODProf'] > 0) & (df['Prof'] > df['sellEODProf']), 'movement'] = "\\\\"



df.tail(20)




,Open,High,Low,Close,Adj Close,Volume,ClosePrev,OpenNext,Prof,openProfPercent,isHighMoreThanClose,best,worked,cross,sellEODProf,% Change,strat%,isBelowPrevClose,Action
Date,,,,,,,,,,,,,,,,,,,
2021-07-19,3532.580078,3550.209961,3499.159912,3549.590088,3549.590088,3784600,3573.629883,3567.320068,-41.049805,-114.868652,False,-65.535387,False,False,-67.269963,-0.672700,-67.269963,True,BuyAtOpen
2021-07-20,3567.320068,3592.000000,3518.000000,3573.189941,3573.189941,3255700,3549.590088,3576.379883,17.729980,49.949375,True,119.478337,True,False,66.486138,0.664861,49.949375,True,BuyAtOpen
2021-07-21,3576.379883,3586.449951,3543.639893,3585.199951,3585.199951,2319000,3573.189941,3587.229980,3.189941,8.927433,True,37.109725,True,False,33.611451,0.336115,8.927433,True,BuyAtOpen
2021-07-22,3587.229980,3640.020020,3582.270020,3638.030029,3638.030029,3265400,3585.199951,3640.000000,2.030029,5.662248,True,152.906586,True,False,147.356016,1.473560,5.662248,True,BuyAtOpen
2021-07-23,3640.000000,3666.110107,3622.040039,3656.639893,3656.639893,2436300,3638.030029,3673.169922,1.969971,5.414938,True,77.184844,True,False,51.153682,0.511537,5.414938,True,BuyAtOpen
2021-07-26,3673.169922,3712.080078,3647.250000,3699.820068,3699.820068,2900100,3656.639893,3698.500000,16.530029,45.205516,True,151.615109,True,False,118.087034,1.180870,45.205516,True,BuyAtOpen
2021-07-27,3698.500000,3698.500000,3586.149902,3626.389893,3626.389893,4131900,3699.820068,3633.780029,-1.320068,-3.567926,False,-3.567926,False,False,-198.469586,-1.984696,-198.469586,True,BuyAtClose
2021-07-28,3633.780029,3658.419922,3601.000000,3630.320068,3630.320068,2999400,3626.389893,3627.750000,7.390137,20.378770,True,88.324836,True,False,10.837709,0.108377,20.378770,True,NA
2021-07-29,3627.750000,3637.949951,3580.010010,3599.919922,3599.919922,5520000,3630.320068,3347.949951,-2.570068,-7.079454,True,21.017108,True,True,-83.739576,-0.837396,0.000000,True,NA


In [6]:
x = df['% Change'].to_numpy()
tmp = [0]
summ = 0
ba = 0
for i in range(1, len(x)):
    if summ == 0:
        summ = x[i]
    else:
        if x[i]*x[i-1] < 0:
            summ = x[i]
        else:
            summ += x[i]
    ba += x[i]
    tmp.append(summ)
    print(ba)
dff = pd.DataFrame(tmp,columns=['CumuChange'])
dff.set_index(df.index, inplace=True)
dff.tail(50)

df = pd.concat([df, dff], axis=1)
df.tail(20)

-3.0720563459048753
-2.024570787971971
-4.256952357717377
-3.9545979514140908
-2.0115166566402323
-0.5379992756058589
-1.7032995771918298
-1.7181491714225132
-1.226785764881995
-2.600069292027312
-1.2916439299643074
-0.8583588154936004
-0.670885067324346
-1.7443334015854681
-1.9622838375883769
-2.099425318284606
-1.6228252419776212
-3.07551930134746
-2.472761202733496
-2.728824760833646
-0.6619113536084344
-0.13987642337951167
1.947806310989617
1.863623656056725
2.9703436925651516
2.9484683146882005
3.8978884759028674
6.064347453558236
5.997281619818671
5.052604619365944
6.543067191340198
6.49685692746601
4.934562369914532
3.553900972995403
4.801304503476197
4.924711468639604
4.693282909551083
4.4842827005508745
6.7566594015229295
11.449366585333983
12.01632964436407
12.958547403705607
12.635081890587353
12.61384051325076
11.506152278175621
11.623623074958115
10.252510392540467
8.667082183311841
7.994382555476982
8.659243931897192
8.995358439174082
10.468918598844722
10.980455423208063

,Open,High,Low,Close,Adj Close,Volume,ClosePrev,OpenNext,Prof,openProfPercent,isHighMoreThanClose,best,worked,cross,sellEODProf,% Change,strat%,isBelowPrevClose,Action,CumuChange
Date,,,,,,,,,,,,,,,,,,,,
2021-07-19,3532.580078,3550.209961,3499.159912,3549.590088,3549.590088,3784600,3573.629883,3567.320068,-41.049805,-114.868652,False,-65.535387,False,False,-67.269963,-0.672700,-67.269963,True,BuyAtOpen,-3.629241
2021-07-20,3567.320068,3592.000000,3518.000000,3573.189941,3573.189941,3255700,3549.590088,3576.379883,17.729980,49.949375,True,119.478337,True,False,66.486138,0.664861,49.949375,True,BuyAtOpen,0.664861
2021-07-21,3576.379883,3586.449951,3543.639893,3585.199951,3585.199951,2319000,3573.189941,3587.229980,3.189941,8.927433,True,37.109725,True,False,33.611451,0.336115,8.927433,True,BuyAtOpen,1.000976
2021-07-22,3587.229980,3640.020020,3582.270020,3638.030029,3638.030029,3265400,3585.199951,3640.000000,2.030029,5.662248,True,152.906586,True,False,147.356016,1.473560,5.662248,True,BuyAtOpen,2.474536
2021-07-23,3640.000000,3666.110107,3622.040039,3656.639893,3656.639893,2436300,3638.030029,3673.169922,1.969971,5.414938,True,77.184844,True,False,51.153682,0.511537,5.414938,True,BuyAtOpen,2.986073
2021-07-26,3673.169922,3712.080078,3647.250000,3699.820068,3699.820068,2900100,3656.639893,3698.500000,16.530029,45.205516,True,151.615109,True,False,118.087034,1.180870,45.205516,True,BuyAtOpen,4.166943
2021-07-27,3698.500000,3698.500000,3586.149902,3626.389893,3626.389893,4131900,3699.820068,3633.780029,-1.320068,-3.567926,False,-3.567926,False,False,-198.469586,-1.984696,-198.469586,True,BuyAtClose,-1.984696
2021-07-28,3633.780029,3658.419922,3601.000000,3630.320068,3630.320068,2999400,3626.389893,3627.750000,7.390137,20.378770,True,88.324836,True,False,10.837709,0.108377,20.378770,True,NA,0.108377
2021-07-29,3627.750000,3637.949951,3580.010010,3599.919922,3599.919922,5520000,3630.320068,3347.949951,-2.570068,-7.079454,True,21.017108,True,True,-83.739576,-0.837396,0.000000,True,NA,-0.837396


In [7]:
df[['worked','openProfPercent', 'best', 'sellEODProf', 'strat%', 'cross', '% Change', 'CumuChange', 'Action']].tail(60)

,worked,openProfPercent,best,sellEODProf,strat%,cross,% Change,CumuChange,Action
Date,,,,,,,,,
2021-05-20,True,38.987107,86.267351,49.136341,38.987107,False,0.491363,0.491363,BuyAtOpen
2021-05-21,True,7.143772,27.742912,-137.328353,7.143772,False,-1.373284,-1.373284,BuyAtClose
2021-05-24,True,38.774934,171.303469,130.842536,38.774934,False,1.308425,1.308425,BuyAtOpen
2021-05-25,True,66.810473,107.334932,43.328511,66.810473,False,0.433285,1.741710,BuyAtClose
2021-05-26,True,47.682726,112.547924,18.747375,47.682726,False,0.187474,1.929184,NA
2021-05-27,False,-28.053487,-14.700060,-107.344833,-107.344833,False,-1.073448,-1.073448,BuyAtClose
2021-05-28,True,36.809558,55.353787,-21.795044,36.809558,False,-0.217950,-1.291399,BuyAtClose
2021-06-01,True,63.386558,86.594184,-13.714148,63.386558,False,-0.137141,-1.428540,BuyAtClose
2021-06-02,True,13.826279,50.798000,47.660008,13.826279,False,0.476600,0.476600,NA


In [14]:
# summ = sum(df['actual'])
sellEODProfReturn = df['sellEODProf'].sum() # Similar to holding the stock
sellEODProfReturn

88.63578968244835

In [15]:
sellSODProfReturn = df['strat%'].sum() # More consistent positive returns (Stocks which tend to go up pre market are successful eg. COIN coinbase)
sellSODProfReturn

-290.5899035377929

In [16]:
x = df.tail(1)['Close']
y = df.head(1)['Close']
x,y

(Date
 2021-08-13    3293.969971
 Name: Close, dtype: float64,
 Date
 2021-05-07    3291.610107
 Name: Close, dtype: float64)

In [17]:
100*((x.iloc[0]-y.iloc[0])/x.iloc[0])

0.07164191848252545

In [19]:
sellEODProfReturn/1000

0.08863578968244835

In [20]:
sellSODProfReturn/1000

-0.2905899035377929

In [ ]:
def isCumuChangeNegative():
    
    return True